In [12]:
pip install requests

In [7]:
ACCESS_TOKEN = '7b6EMEqfVSPomwNr3vUaV1ZpeSILZKVAXUpvKZBLxMcRPJtu3dXlDTs5EQyi'
r = requests.get('https://zenodo.org/api/deposit/depositions',
                  params={'access_token': ACCESS_TOKEN})
r.status_code
# 200
r.json()
# []

[{'created': '2024-09-24T14:13:20.883295+00:00',
  'modified': '2024-09-24T14:13:21.330609+00:00',
  'id': 13834155,
  'conceptrecid': '13834154',
  'metadata': {'title': 'Mon dépôt de données',
   'publication_date': '2024-09-24',
   'description': 'Description du jeu de données',
   'access_right': 'open',
   'creators': [{'name': 'Nom, Prénom', 'affiliation': 'Votre affiliation'}],
   'license': 'cc-zero',
   'imprint_publisher': 'Zenodo',
   'upload_type': 'dataset',
   'prereserve_doi': {'doi': '10.5281/zenodo.13834155', 'recid': 13834155}},
  'title': 'Mon dépôt de données',
  'links': {'self': 'https://zenodo.org/api/deposit/depositions/13834155',
   'html': 'https://zenodo.org/deposit/13834155',
   'badge': 'https://zenodo.org/badge/doi/.svg',
   'files': 'https://zenodo.org/api/deposit/depositions/13834155/files',
   'latest_draft': 'https://zenodo.org/api/deposit/depositions/13834155',
   'latest_draft_html': 'https://zenodo.org/deposit/13834155',
   'publish': 'https://zenod

In [5]:
import requests
import pandas as pd

ACCESS_TOKEN = '7b6EMEqfVSPomwNr3vUaV1ZpeSILZKVAXUpvKZBLxMcRPJtu3dXlDTs5EQyi'
url = 'https://zenodo.org/api/records'

# Paramètres de la requête
params = {
    'q': 'Inria, Institut national de recherche en informatique et en automatique',
    'access_token': ACCESS_TOKEN,
    'sort': 'bestmatch',
    'size': 100,  # Limite à 100 résultats par page
    'publication_date': '*'
}

# Fonction pour récupérer les publications
def get_publications():
    publications = []
    page = 1
    while True:
        params['page'] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            hits = data['hits']['hits']
            if not hits:
                break
            # Extraire les informations demandées
            for record in hits:
                creators = record['metadata'].get('creators', [])
                # Concaténation des informations des créateurs (nom, affiliation, orcid)
                creators_info = ', '.join([f"{creator.get('name', 'N/A')} ({creator.get('affiliation', 'N/A')}, {creator.get('orcid', 'N/A')})" for creator in creators])

                # Institution: Concaténer toutes les affiliations des auteurs, en évitant les None
                institutions = ', '.join([creator.get('affiliation') for creator in creators if creator.get('affiliation')])  # Inclut seulement les affiliations non None ou non vides

                # Récupérer le type de ressource si disponible
                publication_type = record['metadata'].get('resource_type', {}).get('type', 'N/A')

                # Récupération des sujets (si disponibles)
                subjects = record['metadata'].get('subjects', [])
                if subjects:
                    subjects_info = ', '.join([f"{subject.get('term', 'N/A')} (ID: {subject.get('identifier', 'N/A')}, Scheme: {subject.get('scheme', 'N/A')})" for subject in subjects])
                else:
                    subjects_info = 'N/A'

                # Récupérer les keywords (mots-clés) si disponibles
                keywords = record['metadata'].get('keywords', [])
                if keywords:
                    keywords_info = ', '.join(keywords)
                else:
                    keywords_info = 'N/A'

                # Créer l'URL de Zenodo en utilisant l'identifiant de l'enregistrement
                record_id = record['id']
                url_zenodo = f"https://zenodo.org/records/{record_id}"

                publications.append({
                    'title': record['metadata'].get('title', 'N/A'),
                    'publication_date': record['metadata'].get('publication_date', 'N/A'),
                    'doi': record['metadata'].get('doi', 'N/A'),
                    'creators': creators_info,
                    'publication_type': publication_type,
                    'access_right': record['metadata'].get('access_right', 'N/A'),
                    'institution': institutions if institutions else 'N/A',  # Toutes les affiliations
                    'subjects': subjects_info,  # Ajout des sujets formatés correctement
                    'keywords': keywords_info,  # Ajout des mots-clés (keywords)
                    'url_zenodo': url_zenodo  # URL Zenodo pour chaque publication
                })
            page += 1
        else:
            print(f"Erreur lors de la récupération des données: {response.status_code}")
            break
    return publications

# Récupérer les publications et les mettre dans un DataFrame
publications = get_publications()
df_publications = pd.DataFrame(publications)

df_publications


Erreur lors de la récupération des données: 400


,title,publication_date,doi,creators,publication_type,access_right,institution,subjects,keywords,url_zenodo
0,DEFI-COLaF/LADaS: 2024-02-20 - First release,2024-02-20,10.5281/zenodo.10682624,"Clérice, Thibault (Institut national de recher...",dataset,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10682624
1,Interpreting Vegetation-Climate Interaction wi...,2024-04-26,10.5281/zenodo.11072948,"Ricardo, Barros Lourenço (McMaster University,...",presentation,open,McMaster University,N/A,N/A,https://zenodo.org/records/11072948
2,Data-Efficient French Language Modeling with C...,2023-08-02,10.18653/v1/2023.findings-acl.320,"Antoun, Wissam (Institut national de recherche...",publication,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10024064
3,Towards a Robust Detection of Language Model G...,2023-07-12,10.5281/zenodo.10024578,"Seddah, Djamé (Institut national de recherche ...",publication,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10024578
4,Enriching the NArabizi Treebank: A Multifacete...,2023-06-10,10.18653/v1/2023.law-1.26,"Riabi, Arij (Institut national de recherche en...",publication,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10024092
...,...,...,...,...,...,...,...,...,...,...
9995,Caractérisation de la pêche récifale par l'ana...,2025-02-06,10.5281/zenodo.14824404,"Behivoke, Faustinato (Institut Halieutique et ...",publication,open,Institut Halieutique et des Sciences Marines (...,N/A,"GPS, forêt aléatoire, apprentissage, effort de...",https://zenodo.org/records/14824404
9996,"FIG. 7 in Le genre Eustra Schmidt-Goebel, 1846...",2001-12-31,10.5281/zenodo.5391043,"Deuve, Thierry (Muséum national d'Histoire nat...",image,open,"Muséum national d'Histoire naturelle, Laborato...",N/A,"Biodiversity, Taxonomy, Animalia, Arthropoda, ...",https://zenodo.org/records/5391043
9997,LA PNEUMOPATHIE INTERSTITIELLE DIFFUSE AU COUR...,2024-06-15,10.21474/IJAR01/18809,"Amal El-Ouakhoumi , Jaouad Yousfi , Mouna Zahl...",dataset,open,"Service de Medecine Interne, Centre Hospitalie...",N/A,N/A,https://zenodo.org/records/11667071
9998,MARTIALISATION DU CORPS ET DES DIFFERENTES PA...,2023-12-16,10.5281/zenodo.10394925,"TCHAOU, Paterne Djidéwou (Université d'Abomey-...",publication,open,"Université d'Abomey-Calavi, Bénin",N/A,N/A,https://zenodo.org/records/10394925


In [35]:
df_publications.to_excel('data_zenodo_inria_filtre_05112024.xlsx', index=False)

In [37]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side

# Charger le fichier Excel existant
wb = load_workbook('data_zenodo_inria_filtre_05112024.xlsx')
ws = wb.active

# Geler la première ligne et la première colonne
ws.freeze_panes = 'B2'

# Appliquer des styles aux en-têtes
header_font = Font(bold=True, color='FFFFFF')  # Police en gras et blanche
fill = PatternFill(fill_type="solid", fgColor="4F81BD")
thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

for cell in ws[1]:
    cell.font = header_font
    cell.fill = fill
    cell.alignment = Alignment(horizontal="center", vertical="center")  # Centrer le texte
    cell.border = thin_border

# Colorer la première colonne en bleu ciel à partir de la deuxième ligne
sky_blue_fill = PatternFill(fill_type="solid", fgColor="87CEEB")  # Code couleur pour le bleu ciel

for row in range(2, ws.max_row + 1):  # Commencer à la deuxième ligne
    ws.cell(row=row, column=1).fill = sky_blue_fill  # Colorer la première colonne

# Ajuster la largeur des colonnes, maximum à 60
for column_cells in ws.columns:
    max_length = max(len(str(cell.value)) for cell in column_cells)
    column_letter = column_cells[0].column_letter
    adjusted_width = min(max_length + 2, 60)  # Limiter la largeur à 60
    ws.column_dimensions[column_letter].width = adjusted_width

    # Appliquer le retour à la ligne automatique, alignement au centre et mettre en gras
    for cell in column_cells:
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)  # Centrer et retour à la ligne
        cell.font = Font(bold=True)  # Mettre en gras

# Ajouter des hyperliens dans la colonne 'urel_hal'
urel_hal_col_idx = None
for idx, cell in enumerate(ws[1], start=1):
    if cell.value == 'url_zenodo':  # Vérifiez si c'est la colonne 'urel_hal'
        urel_hal_col_idx = idx
        break

if urel_hal_col_idx:
    for row in range(2, ws.max_row + 1):  # Commencez à la deuxième ligne
        cell = ws.cell(row=row, column=urel_hal_col_idx)
        if cell.value:  # Assurez-vous qu'il y a une valeur dans la cellule
            cell.hyperlink = cell.value  # Ajoutez le lien hypertexte
            cell.style = 'Hyperlink'  # Appliquez le style d'hyperlien
            cell.font = Font(underline='single', color='0000FF')  # Couleur et soulignement des hyperliens

# Ajouter des filtres à la première ligne
ws.auto_filter.ref = ws.dimensions

# Enregistrer le fichier Excel modifié
wb.save('data_zenodo_inria_filtre_05112024.xlsx')


In [3]:
import requests
import pandas as pd
import re

ACCESS_TOKEN = '7b6EMEqfVSPomwNr3vUaV1ZpeSILZKVAXUpvKZBLxMcRPJtu3dXlDTs5EQyi'
url = 'https://zenodo.org/api/records'

# Paramètres de la requête
params = {
    'q': 'INRIA',
    'access_token': ACCESS_TOKEN,
    'sort': 'bestmatch',
    'size': 100,  # Limite à 100 résultats par page
    'publication_date': '*'
}

# Fonction pour récupérer les publications
def get_publications():
    publications = []
    page = 1
    while True:
        params['page'] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            hits = data['hits']['hits']
            if not hits:
                break
            # Extraire les informations demandées
            for record in hits:
                creators = record['metadata'].get('creators', [])
                # Concaténation des informations des créateurs (nom, affiliation, orcid)
                creators_info = ', '.join([f"{creator.get('name', 'N/A')} ({creator.get('affiliation', 'N/A')}, {creator.get('orcid', 'N/A')})" for creator in creators])

                # Institution: Concaténer toutes les affiliations des auteurs, en évitant les None
                institutions = ', '.join([creator.get('affiliation') for creator in creators if creator.get('affiliation')])  # Inclut seulement les affiliations non None ou non vides

                # Récupérer le type de ressource si disponible
                publication_type = record['metadata'].get('resource_type', {}).get('type', 'N/A')

                # Récupération des sujets (si disponibles)
                subjects = record['metadata'].get('subjects', [])
                if subjects:
                    subjects_info = ', '.join([f"{subject.get('term', 'N/A')} (ID: {subject.get('identifier', 'N/A')}, Scheme: {subject.get('scheme', 'N/A')})" for subject in subjects])
                else:
                    subjects_info = 'N/A'

                # Récupérer les keywords (mots-clés) si disponibles
                keywords = record['metadata'].get('keywords', [])
                if keywords:
                    keywords_info = ', '.join(keywords)
                else:
                    keywords_info = 'N/A'

                publications.append({
                    'title': record['metadata'].get('title', 'N/A'),
                    'publication_date': record['metadata'].get('publication_date', 'N/A'),
                    'doi': record['metadata'].get('doi', 'N/A'),
                    'creators': creators_info,
                    'publication_type': publication_type,
                    'access_right': record['metadata'].get('access_right', 'N/A'),
                    'institution': institutions if institutions else 'N/A',  # Toutes les affiliations
                    'subjects': subjects_info,  # Ajout des sujets formatés correctement
                    'keywords': keywords_info  # Ajout des mots-clés (keywords)
                })
            page += 1
        else:
            print(f"Erreur lors de la récupération des données: {response.status_code}")
            break
    return publications

# Fonction pour filtrer les publications selon le nom de l'institution
# def filter_publications(df):
#     # Expression régulière pour détecter "INRIA", "Inria", "inria", etc.
#     regex_inria = re.compile(r'inria|institut national de recherche en sciences et technologies du numérique','National Institute for Research in Digital Science and Technology', re.IGNORECASE)
    
#     # Filtrer les lignes où l'institution contient une des variantes de "INRIA"
#     filtered_df = df[df['institution'].apply(lambda x: bool(regex_inria.search(x)) if isinstance(x, str) else False)]
    
#     return filtered_df

# Récupérer les publications et les mettre dans un DataFrame
publications = get_publications()
df_publications = pd.DataFrame(publications)

# Filtrer le DataFrame pour ne garder que les institutions qui mentionnent "INRIA" (ou variantes)
df_filtered = filter_publications(df_publications)

# Afficher les premières lignes filtrées
df_filtered


NameError: name 'filter_publications' is not defined

In [108]:
df_filtered_describe = df_filtered.describe(include='all')
df_filtered_describe.to_excel('data_zenodo_describe_21_12_2024.xlsx')
df_filtered_describe

,title,publication_date,doi,creators,publication_type,access_right,institution,subjects,keywords
count,1415,1415,1415,1415,1415,1415,1415,1415,1415
unique,1397,873,1415,1104,12,3,978,2,721
top,Computational modelling of the cellular interp...,2023-04-27,10.4122/1.1000000255,"de Freitas Saldanha, Raphael (Research Centre ...",dataset,open,Research Centre Inria Sophia Antipolis - Médit...,N/A,N/A
freq,4,31,1,78,516,1364,162,1414,603


In [104]:
df_filtered.to_excel('data_zenodo_21_12_2024.xlsx')

In [116]:
import requests
import pandas as pd
import re

ACCESS_TOKEN = '7b6EMEqfVSPomwNr3vUaV1ZpeSILZKVAXUpvKZBLxMcRPJtu3dXlDTs5EQyi'
url = 'https://zenodo.org/api/records'

# Paramètres de la requête
params = {
    'q': 'INRIA',
    'access_token': ACCESS_TOKEN,
    'sort': 'bestmatch',
    'size': 100,  # Limite à 100 résultats par page
    'publication_date': '*'
}

# Fonction pour récupérer les publications
def get_publications():
    publications = []
    page = 1
    while True:
        params['page'] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            hits = data['hits']['hits']
            if not hits:
                break
            # Extraire les informations demandées
            for record in hits:
                # Informations générales qui restent constantes pour tous les créateurs
                title = record['metadata'].get('title', 'N/A')
                publication_date = record['metadata'].get('publication_date', 'N/A')
                doi = record['metadata'].get('doi', 'N/A')
                publication_type = record['metadata'].get('resource_type', {}).get('type', 'N/A')
                access_right = record['metadata'].get('access_right', 'N/A')
                subjects = record['metadata'].get('subjects', [])
                subjects_info = ', '.join([f"{subject.get('term', 'N/A')} (ID: {subject.get('identifier', 'N/A')}, Scheme: {subject.get('scheme', 'N/A')})" for subject in subjects]) if subjects else 'N/A'
                keywords = record['metadata'].get('keywords', [])
                keywords_info = ', '.join(keywords) if keywords else 'N/A'

                # Champs supplémentaires
                version = record['metadata'].get('version', 'N/A')
                citation = record.get('citation_count', 'N/A')
                grant = record['metadata'].get('grants', [])
                grant_info = ', '.join([grant_item.get('id', 'N/A') for grant_item in grant]) if grant else 'N/A'
                relation = record['metadata'].get('related_identifiers', [])
                relation_info = ', '.join([f"{rel.get('identifier', 'N/A')} ({rel.get('relation', 'N/A')})" for rel in relation]) if relation else 'N/A'
                publication_status = record['metadata'].get('publication_status', 'N/A')
                communities = record['metadata'].get('communities', [])
                communities_info = ', '.join([community.get('title', 'N/A') for community in communities]) if communities else 'N/A'

                # Pour chaque créateur, créer une ligne distincte
                creators = record['metadata'].get('creators', [])
                for creator in creators:
                    name = creator.get('name', 'N/A')
                    institution = creator.get('affiliation', 'N/A')
                    orcid = creator.get('orcid', 'N/A')

                    publications.append({
                        'title': title,
                        'publication_date': publication_date,
                        'doi': doi,
                        'publication_type': publication_type,
                        'access_right': access_right,
                        'subjects': subjects_info,
                        'keywords': keywords_info,
                        'version': version,
                        'citation': citation,
                        'grant': grant_info,
                        'relation': relation_info,
                        'publication_status': publication_status,
                        'communities': communities_info,
                        'creator_name': name,  # Nom et prénom du créateur
                        'creator_institution': institution,  # Institution du créateur
                        'creator_orcid': orcid  # ORCID du créateur
                    })
            page += 1
        else:
            print(f"Erreur lors de la récupération des données: {response.status_code}")
            break
    return publications

# Fonction pour filtrer les publications selon le nom de l'institution
def filter_publications(df):
    regex_inria = re.compile(r'inria|institut national de recherche en sciences et technologies du numérique', re.IGNORECASE)
    filtered_df = df[df['creator_institution'].apply(lambda x: bool(regex_inria.search(x)) if isinstance(x, str) else False)]
    return filtered_df

# Récupérer les publications et les mettre dans un DataFrame
publications = get_publications()
df_publications = pd.DataFrame(publications)

# Filtrer le DataFrame pour ne garder que les institutions qui mentionnent "INRIA" (ou variantes)
df_filtered = filter_publications(df_publications)

# Afficher les premières lignes filtrées
df_filtered


,title,publication_date,doi,publication_type,access_right,subjects,keywords,version,citation,grant,relation,publication_status,communities,creator_name,creator_institution,creator_orcid
10,Inria-Empenn/EEG_preprocessing: Revision 1,2022-10-12,10.5281/zenodo.7189958,software,open,N/A,N/A,revised_1,N/A,N/A,https://github.com/Inria-Empenn/EEG_preprocess...,N/A,N/A,Camille Maumet,Inria,N/A
16,Sensitivity Analysis for the Numerical Simulat...,2006-06-18,10.4122/1.1000000250,publication,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"N/A, N/A","marchand, estelle",inria rocquencourt,N/A
17,Sensitivity Analysis for the Numerical Simulat...,2006-06-18,10.4122/1.1000000250,publication,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"N/A, N/A","clement, francois",inria rocquencourt,N/A
18,Newton-Krylov methods for coupling transport w...,2006-06-18,10.4122/1.1000000255,publication,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"N/A, N/A","Kern, Michel",INRIA,N/A
19,Newton-Krylov methods for coupling transport w...,2006-06-18,10.4122/1.1000000255,publication,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"N/A, N/A","Amir, Laila",ITASCA / INRIA,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9548,Processing Encrypted Data Using Homomorphic En...,2017-09-22,10.5281/zenodo.2594695,publication,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"Minelli, Michele",INRIA and D´epartement d'informatique de l'ENS...,N/A
9560,"Dataset of manuscript ""Automatic fault mapping...",2020-10-28,10.5281/zenodo.4611494,dataset,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"Tasar, Onur","Université Côte d'Azur, Inria, Sophia Antipoli...",N/A
9561,"Dataset of manuscript ""Automatic fault mapping...",2020-10-28,10.5281/zenodo.4611494,dataset,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"Girard, Nicolas","Université Côte d'Azur, Inria, Sophia Antipoli...",N/A
9570,Experimental data of: When higher carrying cap...,2018-09-17,10.5281/zenodo.1420074,dataset,open,N/A,Trichogramma chilonis expansion,N/A,N/A,N/A,10.1101/307322 (isSupplementTo),N/A,N/A,Ludovic Mailleret,"Université Côte d'Azur, INRA, CNRS, ISA, Franc...",N/A


In [118]:
df_filtered.describe(include='all')

,title,publication_date,doi,publication_type,access_right,subjects,keywords,version,citation,grant,relation,publication_status,communities,creator_name,creator_institution,creator_orcid
count,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308,2308
unique,1397,873,1415,12,3,2,721,214,1,5,419,1,4,1199,574,380
top,nilearn,2023-04-27,10.5281/zenodo.10948303,dataset,open,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,"de Freitas Saldanha, Raphael",Inria,N/A
freq,14,63,14,780,2215,2306,965,1453,2308,2243,1633,2308,1937,80,364,1370


In [146]:
df_filtered['version'].value_counts()

version
N/A       1453
1.0        119
1.0.0       68
1           45
v1.0.0      36
          ... 
0.5          1
v.3.1        1
v0.4.0       1
0.1.3        1
2.14.4       1
Name: count, Length: 214, dtype: int64